### Import Files

In [25]:
import numpy as np
import pandas as pd
import torch
import os
from tqdm import tqdm
from sklearn.decomposition import PCA
from sklearn.cluster import AgglomerativeClustering
from torch.utils.data import Dataset, DataLoader
from transformers import BertModel, BertTokenizer
from sklearn.cluster import DBSCAN
import math
from sklearn.manifold import TSNE
import string
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [26]:
path=os.getcwd()
path=os.path.join(path, "rubert_cased")
tokenizer = BertTokenizer.from_pretrained(path)
model = BertModel.from_pretrained(path, output_hidden_states = True)

In [27]:
# Сброс ограничений на количество выводимых рядов
pd.set_option('display.max_rows', None)
 
# Сброс ограничений на число столбцов
pd.set_option('display.max_columns', None)
 
# Сброс ограничений на количество символов в записи
pd.set_option('display.max_colwidth', None)

### Get DataBase

In [28]:
name="Amalia"
df = pd.read_csv(f'./chats/{name}/ml_{name}.csv')  
text_messages = df['text']  

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('universal_tagset')

stop_words = set(stopwords.words('russian'))

def preprocess_text(text):
    tokens = word_tokenize(text)
    tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() not in stop_words and word not in string.punctuation]
    # tokens = [word.lower() for word in tokens if word.isalpha() and word.lower() and word not in string.punctuation]
    return ' '.join(tokens)

df['cleaned_text'] = df['text'].apply(preprocess_text)

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\Mchomak\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [29]:
text=df["cleaned_text"]

In [30]:
text[0]

'парень выпал дед сносит'

### Tokenize data|

In [31]:
class CustomDataset(Dataset):
    # инициализируем полученные данные
    def __init__(self, X):
        self.text = X
    # функция, которая превращает текст в токен pytorch
    def tokenize(self, text):
        return tokenizer(text, return_tensors='pt', padding='max_length', truncation=True, max_length=150)
    # возвращает длину текста
    def __len__(self):
        return self.text.shape[0]
    # по полученному индексу берет текст, токенизирует его и возвращает словарь с ним
    def __getitem__(self, index):
        output = self.text[index]
        output = self.tokenize(output)
        return {k: v.reshape(-1) for k, v in output.items()}


eval_ds = CustomDataset(text)
eval_dataloader = DataLoader(eval_ds, batch_size=10)

### Get Embedвings

In [32]:
# %pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118

In [33]:
torch.cuda.is_available()

True

In [34]:
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output['last_hidden_state']
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    sum_embeddings = torch.sum(token_embeddings * input_mask_expanded, 1)
    sum_mask = torch.clamp(input_mask_expanded.sum(1), min=1e-9)
    return sum_embeddings / sum_mask

In [35]:
device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")
model.to(device)
# переводит модель в режим оценки
model.eval()

# создает пустое вложение куда будут сохранятся данные
embeddings = torch.Tensor().to(device)

with torch.no_grad():
    for n_batch, batch in enumerate(tqdm(eval_dataloader)):
        # помещает каждый тензор в нужное место, gpu или cpu 
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        embeddings = torch.cat([embeddings, mean_pooling(outputs, batch['attention_mask'])])
    embeddings = embeddings.cpu().numpy()

  0%|          | 0/7 [00:00<?, ?it/s]

100%|██████████| 7/7 [00:00<00:00, 22.18it/s]


### Clastering text

In [36]:
clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=0.4, affinity='cosine', linkage='average').fit(embeddings)

c:\Users\Mchomak\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\cluster\_agglomerative.py:1006: FutureWarning:

Attribute `affinity` was deprecated in version 1.2 and will be removed in 1.4. Use `metric` instead



In [37]:
# dbscan = DBSCAN(eps=0.1, min_samples=5)
# df['cluster'] = dbscan.fit_predict(embeddings)

In [38]:
# pca = PCA(n_components=2, random_state=42)
# emb_2d = pd.DataFrame(pca.fit_transform(embeddings), columns=['x', 'y'])

In [39]:
tsne = TSNE(n_components=2, random_state=42)
emb_2d = tsne.fit_transform(embeddings)

In [40]:
df["x"]=emb_2d[:, 0]
df["y"]=emb_2d[:, 1]
df['cluster'] = clustering.labels_

### Visualise clasters

In [41]:
df = df.sort_values(by='cluster')

In [42]:
import plotly.express as px
fig = px.scatter(df, x='x', y='y', color='cluster', width=1000, height=1000,
                 hover_name="text", hover_data=["x", "y"])

fig.update_traces(marker=dict(size=10))

fig.show()

In [43]:
import plotly.graph_objects as go
fig = go.Figure()

for cluster in df['cluster'].unique():
    # cluster=23
    cluster_data = df[df['cluster'] == cluster]
    fig.add_trace(go.Scatter(
        x=cluster_data['x'],
        y=cluster_data['y'],
        mode='markers',
        marker=dict(color=cluster, size=10, line=dict(color='black', width=2)),  
        text=cluster_data['text'],
        name=f'Cluster {cluster}'
        ))

fig.update_layout(
    xaxis_title='X',
    yaxis_title='Y',
    legend=dict(title='Clusters', orientation="h", yanchor="bottom", y=1.02, xanchor="left", x=0.01),
    hoverlabel=dict(bgcolor="white", font_size=12, namelength=-1),
    width=1000, height=1000
)

fig.show()

### Watch all clasters text

In [44]:
df.head(10)

,text,from,cleaned_text,x,y,cluster
58,Будет хилить примерно по,McHomak,хилить примерно,-22.448978,132.210037,0
19,Мммм... Да....\r\n\r\n,Амалия,мммм,-139.562622,31.385120,0
18,Почта,McHomak,почта,56.936821,351.987396,0
34,"Спасибо, что пользуетесь -",Амалия,спасибо пользуетесь,14.243969,-315.684357,1
67,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,300.945099,-562.013123,1
35,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,14.243969,-315.684357,1
37,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,277.279510,-179.494202,1
38,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,277.279510,-179.494202,1
39,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,277.279510,-179.494202,1
40,"Спасибо, что пользуетесь -",McHomak,спасибо пользуетесь,277.279510,-179.494202,1


In [45]:
# количество кластеров
print(f"Total number of clusters: {len(df['cluster'].unique())}")

Total number of clusters: 17


In [46]:
text_distance=pd.DataFrame(columns=["text", "cluster", "distance"])
for cluster_id in df['cluster'].unique():
    cluster_data = df[df['cluster'] == cluster_id]
    x_mean=cluster_data["x"].mean()
    y_mean=cluster_data["y"].mean()

    for index, row in cluster_data.iterrows(): 
        distance=math.sqrt((row["x"]-x_mean)**2 + (row["y"]-y_mean)**2)
        text_distance.loc[len(text_distance.index)] = [row["text"],cluster_id, distance]

In [47]:
for cluster_id in df['cluster'].unique():
    print(f"Claster {cluster_id} has len: {len(df[df['cluster'] == cluster_id])}")

Claster 0 has len: 3
Claster 1 has len: 33
Claster 2 has len: 1
Claster 3 has len: 19
Claster 4 has len: 1
Claster 5 has len: 1
Claster 6 has len: 1
Claster 7 has len: 1
Claster 8 has len: 1
Claster 9 has len: 1
Claster 10 has len: 1
Claster 11 has len: 1
Claster 12 has len: 1
Claster 13 has len: 1
Claster 14 has len: 1
Claster 15 has len: 1
Claster 16 has len: 1


In [48]:
text_distance = text_distance.sort_values(by=["cluster", 'distance'])
text_distance.groupby('cluster').head(5).reset_index(drop= True )

,text,cluster,distance
0,Будет хилить примерно по,0,41.597363
1,Мммм... Да....\r\n\r\n,0,175.104420
2,Почта,0,202.243762
3,"Спасибо, что пользуетесь -",1,132.770508
4,"Спасибо, что пользуетесь -",1,132.770508
5,"Спасибо, что пользуетесь -",1,132.770508
6,"Спасибо, что пользуетесь -",1,132.770508
7,"Спасибо, что пользуетесь -",1,132.770508
8,"Ученики МОУ Кадетская школа \r\nК. Рамиль, К. Сергей, Д. Ярослав и О. Егор стали финалистами регионального трека Всероссийского конкурса научно-технологических проектов «Большие вызовы», в котором приняли участие более 800 школьников Московской области.\r\n\r\nРебята представили проект по направлению «Беспилотный транспорт и логистические системы» по теме «Автономная охрана общеобразовательных учреждений с помощью БПЛА» и стали участниками очной образовательной программы «Взлета», которая пройдёт в кампусе Гимназии Примакова.\r\n\r\nПоздравляем и гордимся нашими учениками и желаем им успеха в покорении следующих вершин!\r\n.\r\n.\r\n.\r\n.\r\n",2,0.000000
9,"Рад был помочь! Ваш,",3,115.386229
